<a href="https://colab.research.google.com/github/Dan2912/projeto_fort_tech/blob/main/ETL/SILVER/etl_bronze_silver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETAPA 0 - INSTALAR BIBLIOTECAS (apenas se necessário)


In [10]:
!pip install -q boto3 s3fs pandas pyarrow


# ETAPA 1 - IMPORTAÇÕES

In [11]:
import os
from datetime import datetime
from getpass import getpass
import pandas as pd
import s3fs

# ETAPA 2 - DEFINIR PARÂMETROS E CONEXÃO AO S3

In [12]:
bucket = "data-lake-eletro-fort"
bronze_prefix = "bronze/base_vendas/"
silver_prefix = "silver/base_vendas/"

# Subpastas da Bronze a serem unificadas
pastas_bronze = [
    f"{bucket}/{bronze_prefix}carga_inicial/",
    f"{bucket}/{bronze_prefix}data=2025-06-21/"
]

# Credenciais AWS
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID") or getpass("🔐 AWS Access Key ID: ")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY") or getpass("🔐 AWS Secret Access Key: ")

fs = s3fs.S3FileSystem(key=aws_access_key, secret=aws_secret_key)


🔐 AWS Access Key ID: ··········
🔐 AWS Secret Access Key: ··········


# ETAPA 3 - LER PARQUETS DAS DUAS PASTAS E UNIFICAR

In [7]:
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID") or getpass("🔐 AWS Access Key ID: ")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY") or getpass("🔐 AWS Secret Access Key: ")

fs = s3fs.S3FileSystem(key=aws_access_key, secret=aws_secret_key)


🔐 AWS Access Key ID: ··········
🔐 AWS Secret Access Key: ··········


# ETAPA 3 - LISTAR E LER PARQUETS DAS DUAS PASTAS

In [13]:
todos_arquivos = []
for pasta in pastas_bronze:
    arquivos = fs.glob(f"{pasta}*.parquet")
    print(f"📁 {pasta} ➜ {len(arquivos)} arquivos encontrados.")
    todos_arquivos.extend(arquivos)

if not todos_arquivos:
    raise SystemExit("❌ Nenhum arquivo Parquet encontrado nas pastas especificadas.")

# Unir todos os arquivos em um DataFrame
df_bronze = pd.concat(
    [pd.read_parquet(f"s3://{arquivo}", filesystem=fs) for arquivo in todos_arquivos],
    ignore_index=True
)
print(f"\n📊 Total de registros unificados: {len(df_bronze):,}")


📁 data-lake-eletro-fort/bronze/base_vendas/carga_inicial/ ➜ 1 arquivos encontrados.
📁 data-lake-eletro-fort/bronze/base_vendas/data=2025-06-21/ ➜ 1 arquivos encontrados.

📊 Total de registros unificados: 100


# ETAPA 4 - EXIBIR COLUNAS E ESCOLHER QUAIS LEVAR PARA A SILVER

In [ ]:
print("\n📌 Colunas disponíveis:")
for idx, col in enumerate(df_bronze.columns):
    print(f"{idx:>2}. {col}")

entrada = input("\n✏️ Digite os NÚMEROS ou NOMES das colunas a manter (separadas por vírgula): ")
tokens = [t.strip() for t in entrada.split(",") if t.strip()]

colunas_escolhidas = []
for t in tokens:
    if t.isdigit() and int(t) < len(df_bronze.columns):
        colunas_escolhidas.append(df_bronze.columns[int(t)])
    else:
        colunas_escolhidas.append(t)

# Validar colunas
colunas_escolhidas = [c for c in dict.fromkeys(colunas_escolhidas) if c in df_bronze.columns]
if not colunas_escolhidas:
    raise ValueError("❌ Nenhuma coluna válida foi selecionada.")

print(f"\n✅ Colunas escolhidas: {colunas_escolhidas}")
df_silver = df_bronze[colunas_escolhidas].copy()



📌 Colunas disponíveis:
 0. id_venda
 1. id_produto
 2. produto
 3. modelo
 4. serie
 5. id_vendedor
 6. nome_vendedor
 7. valor_venda
 8. comissao_pct
 9. valor_comissao
10. cpf_cnpj
11. razao_social_cliente
12. data_venda
13. data_entrega
14. data_pgto_comissao
15. transportadora
16. codigo_rastreamento
17. tipo_pagamento
18. prazo_pagamento
19. tipo_venda
20. data


# ETAPA 5 - APLICAR MÁSCARAS EM DADOS SENSÍVEIS

In [15]:
import re

def mascarar_cpf_cnpj(valor: str) -> str:
    valor = re.sub(r'[^0-9]', '', str(valor))  # remove pontuação
    if len(valor) == 11:  # CPF
        return f'***.***.***-{valor[-2:]}'  # ***.***.***-XX
    elif len(valor) == 14:  # CNPJ
        return f'**.***.***/****-{valor[-2:]}'  # **.***.***/****-XX
    else:
        return valor  # retorna original se não for CPF/CNPJ válido

if 'cpf_cnpj' in df_silver.columns:
    df_silver['cpf_cnpj'] = df_silver['cpf_cnpj'].apply(mascarar_cpf_cnpj)
    print("🔒 Coluna 'cpf_cnpj' mascarada com sucesso.")


🔒 Coluna 'cpf_cnpj' mascarada com sucesso.


# ETAPA 6 - VISUALIZAR DADOS FINAIS APÓS MASCARAMENTO

In [16]:
print("\n🧪 Visualização dos dados prontos para a camada Silver:")
display(df_silver.head(10))  # mostra as 10 primeiras linhas



🧪 Visualização dos dados prontos para a camada Silver:


,id_produto,produto,modelo,id_vendedor,nome_vendedor,valor_venda,comissao_pct,valor_comissao,cpf_cnpj,razao_social_cliente,data_venda,tipo_pagamento,prazo_pagamento,tipo_venda
0,4,Monitor LG,UltraGear,5,Lucas Rocha,2574.89,5,128.74,***.***.***-07,Carvalho e Filhos,14/06/2025,Cartão de Débito,3x Sem Juros,Presencial
1,1,Tablet Samsung,Galaxy Tab A7,4,Ana Beatriz,2860.44,5,143.02,***.***.***-61,Fogaça,20/06/2025,Cartão de Débito,À Vista,Online
2,2,Notebook Dell,Inspiron 15,4,Ana Beatriz,8110.66,5,405.53,***.***.***-05,Sales e Filhos,11/06/2025,Boleto,12x com Juros,Presencial
3,4,Monitor LG,UltraGear,4,Ana Beatriz,3332.54,5,166.63,***.***.***-95,Martins S.A.,14/06/2025,Boleto,3x Sem Juros,Online
4,4,Monitor LG,UltraGear,1,João Silva,8648.47,5,432.42,***.***.***-97,Duarte Cavalcanti S.A.,13/06/2025,Cartão de Crédito,À Vista,Presencial
5,2,Notebook Dell,Inspiron 15,2,Maria Oliveira,4781.51,5,239.08,***.***.***-52,Caldeira,18/06/2025,Boleto,12x com Juros,Online
6,5,Fone de Ouvido JBL,Tune 510BT,4,Ana Beatriz,5164.56,5,258.23,***.***.***-51,Caldeira Porto S/A,16/06/2025,Pix,12x com Juros,Online
7,1,Tablet Samsung,Galaxy Tab A7,3,Carlos Souza,3793.77,5,189.69,***.***.***-40,Melo Cardoso - EI,15/06/2025,Cartão de Débito,12x com Juros,Presencial
8,3,Smartphone Apple,iPhone 13 Pro,4,Ana Beatriz,5982.01,5,299.10,***.***.***-30,da Mota,17/06/2025,Cartão de Débito,12x com Juros,Presencial
9,5,Fone de Ouvido JBL,Tune 510BT,5,Lucas Rocha,615.51,5,30.78,***.***.***-05,Dias,12/06/2025,Boleto,3x Sem Juros,Online


# ETAPA 7 - SALVAR DADOS NA CAMADA SILVER

In [17]:
hoje = datetime.today().date().isoformat()
caminho_silver = f"{silver_prefix}data={hoje}/vendas_silver_{hoje}.parquet"

df_silver.to_parquet(
    f"s3://{bucket}/{caminho_silver}",
    index=False,
    engine="pyarrow",
    filesystem=fs
)

print(f"\n🚀 Dados salvos com sucesso em:\ns3://{bucket}/{caminho_silver}")
print("🏁 ETL Bronze ➜ Silver finalizado com sucesso! 🎉")


🚀 Dados salvos com sucesso em:
s3://data-lake-eletro-fort/silver/base_vendas/data=2025-06-25/vendas_silver_2025-06-25.parquet
🏁 ETL Bronze ➜ Silver finalizado com sucesso! 🎉
